In [2]:
#import packages
import pandas as pd
import os
import subprocess

In [25]:
#import scraped data
df = pd.read_csv(str(os. getcwd())+"/../data/AttorneyGeneral.csv")
df = df.where(pd.notnull(df), None)
df = df.dropna(how='all')

In [27]:
#remove the position webs
list = [".gov",".us"]
for i, row in df.iterrows():
    if row.CampWeb is not None:
        if (row.CampWeb[-1] == "/" and 
        row.CampWeb.count("/") > 3) or (row.CampWeb[-1] != "/" and 
        row.CampWeb.count("/") > 2) or (any(item in row.CampWeb for item in list)):
            row.CampWeb = None

In [29]:
# Add a column for election results 
df["Result"]="Lose"
currentS = ""
for i, row in df.iterrows():
    if row.State is not None:
        if row.State != currentS:
            currentS = row.State
            row.Result = "Win"

In [31]:
df

,Names,State,CandPage,CampWeb,CampHand,OffHand,PerHand,Party,Result
0,Steve Marshall,Alabama,https://ballotpedia.org/Steve_Marshall,None,SteveMarshallAL,AGSteveMarshall,None,Republican Party,Win
1,Wendell Major,Alabama,https://ballotpedia.org/Wendell_Major,https://wendellmajor.com/,None,None,None,Democratic Party,Lose
2,Kris Mayes,Arizona,https://ballotpedia.org/Kris_Mayes,https://www.krismayes.com,krismayes,None,None,Democratic Party,Win
3,Abraham Hamadeh,Arizona,https://ballotpedia.org/Abraham_Hamadeh,https://www.abeforag.com,abrahamhamadeh,None,None,Republican Party,Lose
4,Samantha Severson,Arizona,https://ballotpedia.org/Samantha_Severson,None,None,None,None,Libertarian Party,Lose
...,...,...,...,...,...,...,...,...,...
69,Eric Toney,Wisconsin,https://ballotpedia.org/Eric_Toney,https://erictoney.com/,EricJToney,None,None,Republican Party,Lose
70,Leevin Taitano Camacho,Guam,https://ballotpedia.org/Leevin_Taitano_Camacho,http://oagguam.org/,None,oagguam,AGLeevinCamacho,Independent,Win
71,Douglas Moylan,Guam,https://ballotpedia.org/Douglas_Moylan,None,None,None,None,Nonpartisan,Lose
72,Edward Manibusan,Northern Mariana Islands,https://ballotpedia.org/Edward_Manibusan,None,None,None,None,Nonpartisan,Win


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Names     74 non-null     object
 1   State     74 non-null     object
 2   CandPage  74 non-null     object
 3   CampWeb   34 non-null     object
 4   CampHand  52 non-null     object
 5   OffHand   19 non-null     object
 6   PerHand   10 non-null     object
 7   Party     74 non-null     object
dtypes: object(8)
memory usage: 4.8+ KB


In [6]:
text = "https://www.myfloridalegal.com/"
text[-1] == "/"

True

In [19]:
df

,Names,State,CandPage,CampWeb,CampHand,OffHand,PerHand,Party,Result
0,Steve Marshall,Alabama,https://ballotpedia.org/Steve_Marshall,None,SteveMarshallAL,AGSteveMarshall,None,Republican Party,Lose
1,Wendell Major,Alabama,https://ballotpedia.org/Wendell_Major,https://wendellmajor.com/,None,None,None,Democratic Party,Lose
2,Kris Mayes,Arizona,https://ballotpedia.org/Kris_Mayes,https://www.krismayes.com,krismayes,None,None,Democratic Party,Lose
3,Abraham Hamadeh,Arizona,https://ballotpedia.org/Abraham_Hamadeh,https://www.abeforag.com,abrahamhamadeh,None,None,Republican Party,Lose
4,Samantha Severson,Arizona,https://ballotpedia.org/Samantha_Severson,None,None,None,None,Libertarian Party,Lose
...,...,...,...,...,...,...,...,...,...
69,Eric Toney,Wisconsin,https://ballotpedia.org/Eric_Toney,https://erictoney.com/,EricJToney,None,None,Republican Party,Lose
70,Leevin Taitano Camacho,Guam,https://ballotpedia.org/Leevin_Taitano_Camacho,None,None,oagguam,AGLeevinCamacho,Independent,Lose
71,Douglas Moylan,Guam,https://ballotpedia.org/Douglas_Moylan,None,None,None,None,Nonpartisan,Lose
72,Edward Manibusan,Northern Mariana Islands,https://ballotpedia.org/Edward_Manibusan,None,None,None,None,Nonpartisan,Lose


In [13]:
#Globals
COMMAND_TEMPLATE = (
    "gh workflow run "
    "--repo {repo} "
    "generate-report.yml "
    "-f url={link} "
)
REPO = "peiwenf/campaign-access-eval" # forked repo

In [14]:
#run the github actions for a dataframe
for i, row in df.iterrows():
    if row.CampWeb is not None:
        command = COMMAND_TEMPLATE.format(
        repo=REPO,
        link=row.CampWeb,
        )
        proc_resp = subprocess.run(
            command.split(" "),
            check=True,
        )
        print(f"row {i} is done!")

row 0 is done!
row 1 is done!
row 2 is done!
row 3 is done!
row 5 is done!
row 6 is done!
row 8 is done!
row 9 is done!
row 10 is done!
row 11 is done!
row 13 is done!
row 14 is done!
row 15 is done!
row 18 is done!
row 19 is done!
row 20 is done!
row 21 is done!
row 22 is done!
row 23 is done!
row 24 is done!
row 25 is done!
row 26 is done!
row 27 is done!
row 28 is done!
row 29 is done!
row 30 is done!
row 31 is done!
row 32 is done!
row 33 is done!
row 34 is done!
row 35 is done!
row 36 is done!
row 37 is done!
row 38 is done!
row 39 is done!
row 40 is done!
row 42 is done!
row 43 is done!
row 45 is done!
row 46 is done!
row 47 is done!
row 48 is done!
row 49 is done!
row 50 is done!
row 51 is done!
row 52 is done!
row 53 is done!
row 55 is done!
row 56 is done!
row 57 is done!
row 59 is done!
row 60 is done!
row 61 is done!
row 62 is done!
row 63 is done!
row 64 is done!
row 66 is done!
row 67 is done!
row 68 is done!
row 69 is done!
row 70 is done!


In [11]:
#download reports
store_dir = str(os. getcwd())+"/../reports/AttorneyGeneral"
subprocess.run(["gh", "run", "download", "-D", store_dir])

CompletedProcess(args=['gh', 'run', 'download', '-D', '/Users/fpw/Desktop/research/campaign-access-eval/access_eval/2022/scripts/../reports/AttorneyGeneral'], returncode=0)

In [33]:
# delete the current workflow 
user="peiwenf" repo = "campaign-access-eval"; gh api repos/$user/$repo/actions/runs \
--paginate -q '.workflow_runs[] | select(.head_branch != "master") | "\(.id)"' | \
xargs -n1 -I % gh api repos/$user/$repo/actions/runs/% -X DELETE

SyntaxError: invalid syntax (516708068.py, line 2)

In [ ]:
# Delete the current workflows- runs after every dataframe - in terminal

OWNER="peiwenf"
REPO="campaign-access-eval"

# list workflows
subprocess.run(["gh", "api", "-X", "GET", \
"repos/peiwenf/campaign-access-eval/actions/workflows", "|", "jq", ".workflows[]", "|", \
    ".name,.id"])

gh api -X GET repos/peiwenf/campaign-access-eval/actions/workflows | jq '.workflows[] | .name,.id'

# copy the ID of the workflow you want to clear and set it
WORKFLOW_ID=45319078

# list runs
gh api -X GET /repos/$OWNER/$REPO/actions/workflows/$WORKFLOW_ID/runs | jq '.workflow_runs[] | .id'

# delete runs (you'll have to run this multiple times if there's many because of pagination)
gh api -X GET /repos/$OWNER/$REPO/actions/workflows/$WORKFLOW_ID/runs | jq '.workflow_runs[] | .id' | xargs -I{} gh api -X DELETE /repos/$OWNER/$REPO/actions/runs/{}